# Make Resnet

In [65]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tqdm
import time

### 1. Data preprocessing

In [66]:
BATCH_SIZE = 100
LAYER_NM = 18
path_data = './data'

if not os.path.exists(path_data):
    os.mkdir(path_data)

transform_ = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
train_data = datasets.CIFAR10(root=path_data, train=True, download=True, transform=transform_)
test_data = datasets.CIFAR10(root=path_data, train=False, download=True, transform=transform_)

train_loader = DataLoader(train_data,batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(test_data,batch_size=BATCH_SIZE, shuffle=False, num_workers=2, drop_last=True)

# define classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [67]:
print(next(iter(train_data))[0].size())
print(next(iter(train_loader))[0].size())

torch.Size([3, 32, 32])
torch.Size([100, 3, 32, 32])


### 2. Build Model

In [68]:
def conv_kernel_1(in_dim, out_dim, stride=1):
    model = nn.Sequential(
        nn.Conv2d(in_dim, out_dim, kernel_size=1, stride=stride),
        nn.BatchNorm2d(out_dim),
        nn.ReLU()
    )
    return model

def conv_kernel_3(in_dim, out_dim, stride=1):
    model = nn.Sequential(
        nn.Conv2d(in_dim, out_dim, kernel_size=3, stride=stride, padding=1),
        nn.BatchNorm2d(out_dim),
        nn.ReLU()
    )
    return model

def conv_kernel_1_no_act(in_dim, out_dim, stride=1):
    model = nn.Sequential(
        nn.Conv2d(in_dim, out_dim, kernel_size=1, stride=stride),
        nn.BatchNorm2d(out_dim)
    )
    return model

def conv_kernel_3_no_act(in_dim, out_dim, stride=1):
    model = nn.Sequential(
        nn.Conv2d(in_dim, out_dim, kernel_size=3, stride=stride, padding=1),
        nn.BatchNorm2d(out_dim)
    )
    return model

In [69]:
class ResBlock(nn.Module):
    def __init__(self, in_dim, out_dim, down=True, pre_dim=64):
        super().__init__()
        
        self.down = down
        self.activation = nn.ReLU()
        self.conv1 = conv_kernel_3(in_dim, out_dim, stride=1)
        self.conv2 = conv_kernel_3_no_act(in_dim, out_dim)      
        if self.down is True:  
            self.downconv = nn.Conv2d(pre_dim, in_dim, kernel_size=1, stride=2)

    def forward(self, x):
        if self.down is True:
            out = self.downconv(x)
        else:
            out = x
        x = self.conv1(out)
        x = self.conv2(x)
        x += out
        x = self.activation(x)
        return x

In [70]:
class ResNet(nn.Module):
    def __init__(self, model_d, y_dim):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1)
        self.maxpool = nn.MaxPool2d(3, stride=2)
        self.block1 = ResBlock(64, 64, down=False)
        self.block2 = ResBlock(64, 64, down=False)
        self.block3 = ResBlock(128, 128, down=True, pre_dim=64)
        self.block4 = ResBlock(128, 128, down=False, pre_dim=128)
        self.block5 = ResBlock(256, 256, down=True, pre_dim=128)
        self.block6 = ResBlock(256, 256, down=False, pre_dim=256)
        self.block7 = ResBlock(512, 512, down=True, pre_dim=256)
        self.block8 = ResBlock(512, 512, down=False, pre_dim=512)
        self.avgpool = nn.AvgPool2d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc_layer = nn.Linear(512, y_dim)
    
    def forward(self, x):
        # print('start--', x.size())
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.block6(x)
        x = self.block7(x)
        x = self.block8(x)
        # print('before avgpool --', x.size())
        x = self.avgpool(x)
        x = x.view(BATCH_SIZE, -1).contiguous()
        x = self.fc_layer(x)
        return x

### 3. Train

In [71]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 1e-3
model_d = next(iter(train_loader))[0].size()[-2] * next(iter(train_loader))[0].size()[-1]
print(model_d)
y_dim = 10 # output class
model = ResNet(model_d, y_dim).to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

df_loss = pd.DataFrame(columns=['loss'])
epochs = 100
batch_size = len(train_loader)
for i in tqdm.tqdm(range(epochs)):
    running_loss = 0.
    for image, label in train_loader:
        x = image.to(device)
        y = label.to(device)

        optimizer.zero_grad()
        output = model(x)
        loss = loss_func(output, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    df_loss.loc[i, 'loss'] = round(running_loss/batch_size, 4)

1024


100%|██████████| 100/100 [28:30<00:00, 17.10s/it]


In [72]:
df_loss

,loss
0,1.358
1,0.9283
2,0.7262
3,0.6015
4,0.497
...,...
95,0.0121
96,0.0107
97,0.012
98,0.0086


In [73]:
correct = 0
total = 0

with torch.no_grad():
    model.eval()
    for input, label in test_loader:
        x = input.to(device)
        y = label.to(device)
        outputs = model(x)
        # value, pred = torch.max(outputs, 1)
        # pred = torch.argmax(outputs, 1)

        total += y.size(0)
        correct += (torch.argmax(outputs, 1) == y).sum().item()
    print("test accuracy : {}%".format((100 * correct / total)))

test accuracy : 80.39%
